This is a notebook to apply neuroHarmonize: a ComBat-GAM  non-linear allowing algorithm over our data to create neuroHarmonized datasets. It should be run in the `neurogam` environment.

In [ ]:
import os
from neuroHarmonize import harmonizationLearn
import pandas as pd
import numpy as np


In [ ]:
filepath = '../open_work/internal_results/cleaned_pvc2s/' 
filename_mri = os.path.join(filepath,'StrokeMRI_pvc2c.csv') 
filename_top = os.path.join(filepath,'TOP_pvc2c.csv') 
filename_sabre = os.path.join(filepath,'SABRE_pvc2_cleaned.csv') 
filename_insight46 =  os.path.join(filepath,'Insight46_pvc2c.csv') 

In [ ]:
# Now we must clean SABRE down to be like the other datasets, including dropping NAns
MRI = pd.read_csv(filename_mri)
TOP = pd.read_csv(filename_top)
Insight46 = pd.read_csv(filename_insight46)

SABRE =  pd.read_csv(filename_sabre)
coly = TOP.columns
# SABRE = SABRE[coly]
# SABRE= SABRE.dropna()
SABRE.tail(3)

In [ ]:
TOP['site'] = 0
MRI['site'] = 1
MRI.head(3)

# Here we will harmonize just TOP and StrokeMRI

In [ ]:
TOPMRI = pd.concat([TOP, MRI])
TOPMRI.head(3) 

In [ ]:
sex_mapping = {'F':0,'M':1}
TOPMRI = TOPMRI.assign(sex = TOPMRI.sex.map(sex_mapping))
TOPMRI.head(2) 

In [ ]:
TOPMRI.columns

In [ ]:
TOPMRI_features = TOPMRI[[ 'gm_vol', 'wm_vol',
       'csf_vol', 'gm_ivc_ratio', 'gmwm_ivc_ratio', 'wmh_vol', 'wmh_count',
       'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'deepwm_b', 'aca_b', 'mca_b', 'pca_b', 'totalgm_b',]]
TOPMRI_covariates = TOPMRI[['age', 'sex','site']]
TOPMRI_covariates = TOPMRI_covariates.rename(columns={'site': 'SITE'})
TOPMRI_covariates

In [ ]:
TOPMRI_covariates.reset_index()

In [ ]:
TOPMRI_features_array = np.array(TOPMRI_features)
TOPMRI_features_array


In [ ]:
# run harmonization and store the adjusted data
my_model, my_data_adj = harmonizationLearn(TOPMRI_features_array, TOPMRI_covariates)

In [ ]:
my_data_adj

In [ ]:
neuroharmonized_topmri = pd.DataFrame(my_data_adj, columns = ['gm_vol', 'wm_vol',
       'csf_vol', 'gm_ivc_ratio', 'gmwm_ivc_ratio', 'wmh_vol', 'wmh_count',
       'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'deepwm_b', 'aca_b', 'mca_b', 'pca_b', 'totalgm_b'])

In [ ]:
neuroharmonized_topmri =pd.concat([neuroharmonized_topmri, TOPMRI_covariates.reset_index() ], axis=1)
neuroharmonized_topmri = neuroharmonized_topmri.drop('index', axis=1)

In [ ]:
neuroharmonized_topmri = pd.concat([neuroharmonized_topmri, TOPMRI.participant_id.reset_index()], axis=1)
neuroharmonized_topmri = neuroharmonized_topmri.drop('index', axis=1)

In [ ]:
neuroharmonized_topmri